In [ ]:
import pandas as pd
from pathlib import Path

BASE_CLEAN = Path("clean_vote_ver2")
BASE_DUMP  = Path("dump_vote_ver2")

school = pd.read_csv(BASE_CLEAN / "processed_accounts_school.csv")
user   = pd.read_csv(BASE_CLEAN / "processed_accounts_user.csv")
pay    = pd.read_csv(BASE_CLEAN / "processed_accounts_paymenthistory.csv")
uqr    = pd.read_csv(BASE_CLEAN / "processed_userquestionrecord.csv")
group = pd.read_csv(BASE_DUMP / "accounts_group.csv")

In [ ]:
# 공통: id 정규화
def norm_id(s):
    return s.astype(str).str.strip().str.replace(r"\.0$", "", regex=True)

school["id"] = norm_id(school["id"])
user["user_id"] = norm_id(user["user_id"])
user["group_id"] = norm_id(user["group_id"])
pay["user_id"] = norm_id(pay["user_id"])
uqr["user_id"] = norm_id(uqr["user_id"])
uqr["chosen_user_id"] = norm_id(uqr["chosen_user_id"])

# group 컬럼명 케이스 대응
# (보통 dump accounts_group은 school_id가 있음)
if "id" in group.columns: group["id"] = norm_id(group["id"])
if "school_id" in group.columns: group["school_id"] = norm_id(group["school_id"])

In [ ]:
# 1) 노출: 활성 학교 학생수 합
active_school = school[school["is_active_school"] == 1].copy()
exposure_cnt = int(active_school["student_count"].fillna(0).sum())

# 유저에 school_id 붙이기 (user -> group -> school)
# group.id == user.group_id 이어야 함
user_w_school = user.merge(group[["id", "school_id"]].rename(columns={"id": "group_id"}), on="group_id", how="left")
    .merge(active_school[["id"]].rename(columns={"id": "school_id"}), on="school_id", how="inner")

In [ ]:
exposure_cnt

In [ ]:
# 2) 유입: 활성학교 소속 가입 유저 수

# funnel_user 역할: "회원가입한 유저" = user 테이블 전체(가정)
funnel_user = user[["user_id", "group_id"]].copy()

# 활성화된 학교 정보 추가
abcde_user = pd.merge(funnel_user, group[["id", "school_id"]], left_on="group_id", right_on="id", how="left")
abcde_user = pd.merge(abcde_user, school[["id", "is_active_school"]], left_on="school_id", right_on="id", how="left")

# 활성화된 학교를 다니는 유저만 추출
abcde_user = abcde_user[abcde_user["is_active_school"] == 1]

# 유입 유저 수
acq_users = abcde_user["user_id"].dropna().drop_duplicates()
acq_cnt = len(acq_users)

print("유입(회원가입, 활성화학교 필터) 유저 수:", acq_cnt)
abcde_user.head(5)


In [ ]:
abcde_signup_user = abcde_user['user_id'].unique()
len(abcde_signup_user)

In [ ]:
acq_cnt

In [ ]:
# 3) 참여: 유입 유저 중 (투표한 사람 OR 투표받은 사람)
# 참여 유저를 uqr에서 뽑고(acq 유저와 inner join)
eng_a = uqr[["user_id"]].rename(columns={"user_id": "uid"}).dropna()
eng_b = uqr[["chosen_user_id"]].rename(columns={"chosen_user_id": "uid"}).dropna()
eng_all = pd.concat([eng_a, eng_b], ignore_index=True).drop_duplicates()

engaged = eng_all.merge(
    user_w_school[["user_id"]].rename(columns={"user_id": "uid"}),
    on="uid",
    how="inner"
)

eng_cnt = engaged["uid"].nunique()

In [ ]:
eng_cnt

In [ ]:
# 4) 리텐션: 참여 유저 중 활동로그가 하나라도 더 있음
# uqr 기준: 해당 유저 로그 수가 2개 이상
uqr_user_cnt = (
    uqr.groupby("user_id")
       .size()
       .reset_index(name="log_cnt")
)
retained = engaged.merge(
    uqr_user_cnt.rename(columns={"user_id": "uid"}),
    on="uid",
    how="inner"
)
ret_cnt = retained.loc[retained["log_cnt"] >= 2, "uid"].nunique()

In [ ]:
ret_cnt

In [ ]:
# 5) 수익: 하트 구매 로그 있는 유저 
# 리텐션 유저 중 결제기록 존재
rev = retained.loc[retained["log_cnt"] >= 2, ["uid"]].drop_duplicates().merge(
    pay[["user_id"]].rename(columns={"user_id": "uid"}).drop_duplicates(),
    on="uid",
    how="inner"
)
rev_cnt = rev["uid"].nunique()

In [ ]:
rev_cnt

In [ ]:
# ---- 결과 테이블 ----
funnel = pd.DataFrame({
    "funnel": ["노출", "유입(회원가입)", "참여", "리텐션", "수익"],
    "user_counts": [exposure_cnt, acq_cnt, eng_cnt, ret_cnt, rev_cnt],
})
funnel["percent_of_exposure"] = (funnel["user_counts"] / exposure_cnt * 100).round(6)

# 단계별 전환율도 같이 보고 싶으면(이전 단계 대비)
funnel["conv_from_prev"] = (
    funnel["user_counts"] / funnel["user_counts"].shift(1)
).round(6)

display(funnel)
print(f"- 노출({exposure_cnt:,}) → 유입({acq_cnt:,}) → 참여({eng_cnt:,}) → 리텐션({ret_cnt:,}) → 수익({rev_cnt:,})")

In [ ]:
import pandas as pd

BASE_DIR = "clean_vote_ver2"

school   = pd.read_csv(f"{BASE_DIR}/processed_accounts_school.csv")
user     = pd.read_csv(f"{BASE_DIR}/processed_accounts_user.csv")
contacts = pd.read_csv(f"{BASE_DIR}/processed_accounts_user_contacts.csv")
pay      = pd.read_csv(f"{BASE_DIR}/processed_accounts_paymenthistory.csv")
failpay  = pd.read_csv(f"{BASE_DIR}/processed_accounts_failpaymenthistory.csv")
qset     = pd.read_csv(f"{BASE_DIR}/processed_polls_questionset.csv")
uqr      = pd.read_csv(f"{BASE_DIR}/processed_userquestionrecord.csv")

print("로드 완료")

In [ ]:
# 0) id 정규화(교집합 0 방지 핵심)
# user.group_id가 '12.0' 같은 형태였던 이슈 해결

def norm_id(s):
    return s.astype(str).str.strip().str.replace(r"\.0$", "", regex=True)

school["id"] = norm_id(school["id"])
user["user_id"] = norm_id(user["user_id"])
user["group_id"] = norm_id(user["group_id"])
pay["user_id"] = norm_id(pay["user_id"])
failpay["user_id"] = norm_id(failpay["user_id"])
uqr["user_id"] = norm_id(uqr["user_id"])
uqr["chosen_user_id"] = norm_id(uqr["chosen_user_id"])

# created_at이 있으면 초단위로만
if "created_at" in uqr.columns:
    uqr["created_at"] = pd.to_datetime(uqr["created_at"], errors="coerce").dt.floor("s")

In [ ]:
# 1) 유입(Acquisition) = 회원가입 유저
# 지금 데이터 구성상: user 테이블의 유저 = 가입 유저로 간주

acq_users = user[["user_id"]].dropna().drop_duplicates()
acq_cnt = len(acq_users)

In [ ]:
acq_cnt

In [ ]:
# 2) 참여(Engagement) = 유입 유저 중 "투표한 사람" + "투표받은 사람"
# accounts_userquestionrecord: user_id(투표한), chosen_user_id(투표받은)

voter = uqr[["user_id"]].rename(columns={"user_id": "user_id"}).dropna()
chosen = uqr[["chosen_user_id"]].rename(columns={"chosen_user_id": "user_id"}).dropna()

eng_users = (
    pd.concat([voter, chosen], axis=0, ignore_index=True)
      .drop_duplicates()
)

# 유입 기준으로 필터
eng_users = eng_users.merge(acq_users, on="user_id", how="inner")
eng_cnt = len(eng_users)

In [ ]:
eng_cnt

In [ ]:
# 3) 리텐션(Retention) = 참여 유저 중 "활동 로그가 하나라도 더 있는 유저"
#   - 너희 정의: 참여 이후에 ANY 로그 1개라도 더
#   - 내부데이터만 볼 거면, 여기서 '활동 로그'를 무엇으로 볼지 정해야 함
#     (지금 7개 중에서 '추가 활동'으로 가장 자연스러운 후보는 uqr의 추가 기록 / qset의 상태변화 / contacts 변화 등)
#
#   - 일단 가장 보수적으로: uqr에서 참여 유저의 "투표 기록이 2개 이상"을 리텐션으로 잡는 버전
#     (시간 기반 리텐션이 아니라 "추가 활동 1회 이상"을 의미)
# -------------------------
uqr_counts = (
    uqr.groupby("user_id", dropna=True)
       .size()
       .reset_index(name="uqr_cnt")
)

ret_users = (
    eng_users.merge(uqr_counts, on="user_id", how="left")
             .fillna({"uqr_cnt": 0})
)

# "활동 로그가 하나라도 더" = 참여(투표 1회) 이후 추가 1회 => 총 2회 이상
ret_users = ret_users[ret_users["uqr_cnt"] >= 2][["user_id"]].drop_duplicates()
ret_cnt = len(ret_users)

# -------------------------
# 4) 수익(Revenue) = 하트 구매 로그 있는 유저
#   - accounts_paymenthistory에 user_id 존재하면 구매로 간주
# -------------------------
rev_users = pay[["user_id"]].dropna().drop_duplicates()

# 참여 기반 수익(참여한 사람 중 구매자)
rev_from_eng = eng_users.merge(rev_users, on="user_id", how="inner")
rev_cnt = len(rev_from_eng)

# -------------------------
# 5) 퍼널 출력 (유입 기준 / 단계 전환율)
# -------------------------
def pct(a, b):
    return 0 if b == 0 else (a / b * 100)

print("\n--- Funnel (유입 기준) ---")
print(f"유입(회원가입)   : {acq_cnt:,} (100.00%)")
print(f"참여(투표/피투표): {eng_cnt:,} ({pct(eng_cnt, acq_cnt):.2f}%)  / 전환(유입→참여): {pct(eng_cnt, acq_cnt):.2f}%")
print(f"리텐션(추가활동) : {ret_cnt:,} ({pct(ret_cnt, acq_cnt):.2f}%)  / 전환(참여→리텐션): {pct(ret_cnt, eng_cnt):.2f}%")
print(f"수익(구매)       : {rev_cnt:,} ({pct(rev_cnt, acq_cnt):.2f}%)  / 전환(참여→수익): {pct(rev_cnt, eng_cnt):.2f}%")


In [64]:
import pandas as pd

# ====== 경로 ======
CLEAN_DIR = "clean_vote_ver2"
DUMP_DIR  = "dump_vote_ver2"   # "어떤 로그든"을 보려면 dump 쪽 활동 테이블이 필요함

# ====== 로드(전처리된 핵심 7개) ======
school   = pd.read_csv(f"{CLEAN_DIR}/processed_accounts_school.csv")
user     = pd.read_csv(f"{CLEAN_DIR}/processed_accounts_user.csv")
contacts = pd.read_csv(f"{CLEAN_DIR}/processed_accounts_user_contacts.csv")
pay      = pd.read_csv(f"{CLEAN_DIR}/processed_accounts_paymenthistory.csv")
failpay  = pd.read_csv(f"{CLEAN_DIR}/processed_accounts_failpaymenthistory.csv")
qset     = pd.read_csv(f"{CLEAN_DIR}/processed_polls_questionset.csv")
uqr      = pd.read_csv(f"{CLEAN_DIR}/processed_userquestionrecord.csv")

# ====== (선택) dump에서 "활동 로그" 테이블들 로드 ======
# 여기 파일들은 너 dump 폴더에 실제로 있었던 것들 기준
att  = pd.read_csv(f"{DUMP_DIR}/accounts_attendance.csv")         # user_id, created_at 등(있을 가능성 큼)
fr   = pd.read_csv(f"{DUMP_DIR}/accounts_friendrequest.csv")      # send_user_id/receive_user_id, created_at
tlr  = pd.read_csv(f"{DUMP_DIR}/accounts_timelinereport.csv")     # user_id/reported_user_id, created_at
pth  = pd.read_csv(f"{DUMP_DIR}/accounts_pointhistory.csv")       # user_id, created_at (있을 가능성 큼)
blk  = pd.read_csv(f"{DUMP_DIR}/accounts_blockrecord.csv")        # user_id/blocked_id류, created_at (있을 가능성)
wd   = pd.read_csv(f"{DUMP_DIR}/accounts_userwithdraw.csv")       # user_id, created_at (있을 가능성)

print("로드 완료")

# ====== 공통: id 정규화 (교집합 0 방지) ======
def norm_id(s):
    return s.astype(str).str.strip().str.replace(r"\.0$", "", regex=True)

# user_id류 정규화
user["user_id"] = norm_id(user["user_id"])
uqr["user_id"] = norm_id(uqr["user_id"])
uqr["chosen_user_id"] = norm_id(uqr["chosen_user_id"])
pay["user_id"] = norm_id(pay["user_id"])
failpay["user_id"] = norm_id(failpay["user_id"])

# ====== 공통: created_at을 "초 단위"로 (너가 원하는 스타일) ======
def to_dt_floor_s(df, col):
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce").dt.floor("s")
    return df

uqr = to_dt_floor_s(uqr, "created_at")
pay = to_dt_floor_s(pay, "created_at")
failpay = to_dt_floor_s(failpay, "created_at")
att = to_dt_floor_s(att, "created_at")
fr  = to_dt_floor_s(fr,  "created_at")
tlr = to_dt_floor_s(tlr, "created_at")
pth = to_dt_floor_s(pth, "created_at")
blk = to_dt_floor_s(blk, "created_at")
wd  = to_dt_floor_s(wd,  "created_at")

로드 완료


In [65]:
# ====== 1) 참여(Engagement) 유저 정의 ======
# - 투표한(user_id) + 투표받은(chosen_user_id)
voter  = uqr[["user_id", "created_at"]].dropna(subset=["user_id"])
chosen = uqr[["chosen_user_id", "created_at"]].rename(columns={"chosen_user_id": "user_id"}).dropna(subset=["user_id"])

eng = pd.concat([voter, chosen], ignore_index=True)

# 참여 최초 시각(유저별)
eng_first = (
    eng.dropna(subset=["created_at"])
       .groupby("user_id", dropna=True)["created_at"]
       .min()
       .reset_index(name="eng_first_at")
)

eng_users = eng_first[["user_id"]].drop_duplicates()
print("참여 유저 수:", len(eng_users))

참여 유저 수: 15452


In [66]:

# ====== 2) Any-log 후보들을 "한 테이블(activity_log)"로 합치기 ======
# - 어떤 로그든 1개라도 있으면 됨
# - 테이블마다 user 컬럼명이 다를 수 있어, 아래를 실제 컬럼명에 맞춰 자동 대응(있는 것만 사용)

def pick_activity(df, user_cols, time_col, src):
    # user_cols: ['user_id'] 처럼 1개일 수도, ['send_user_id','receive_user_id'] 처럼 2개일 수도
    out_list = []
    if time_col not in df.columns:
        return pd.DataFrame(columns=["user_id", "ts", "src"])
    for uc in user_cols:
        if uc in df.columns:
            tmp = df[[uc, time_col]].rename(columns={uc: "user_id", time_col: "ts"}).copy()
            tmp["src"] = src
            out_list.append(tmp)
    if not out_list:
        return pd.DataFrame(columns=["user_id", "ts", "src"])
    out = pd.concat(out_list, ignore_index=True)
    out["user_id"] = norm_id(out["user_id"])
    return out.dropna(subset=["user_id", "ts"])

activity_parts = []

# 전처리 테이블 중에서도 활동으로 볼 수 있는 것들(결제/실패결제)
activity_parts.append(pick_activity(pay,     ["user_id"], "created_at", "payment"))
activity_parts.append(pick_activity(failpay, ["user_id"], "created_at", "fail_payment"))

# dump 활동 테이블들(출석/친구요청/신고/포인트/차단/탈퇴 등)
activity_parts.append(pick_activity(att, ["user_id"], "created_at", "attendance"))

# friendrequest는 보낸/받은 사람 모두 활동으로 포함
activity_parts.append(pick_activity(fr, ["send_user_id", "receive_user_id"], "created_at", "friendrequest"))

# timelinereport도 신고자/피신고자 모두 로그로 포함(원하면 피신고자는 빼도 됨)
activity_parts.append(pick_activity(tlr, ["user_id", "reported_user_id"], "created_at", "timelinereport"))

activity_parts.append(pick_activity(pth, ["user_id"], "created_at", "pointhistory"))
activity_parts.append(pick_activity(blk, ["user_id", "blocked_user_id"], "created_at", "blockrecord"))
activity_parts.append(pick_activity(wd,  ["user_id"], "created_at", "userwithdraw"))

activity_log = (
    pd.concat(activity_parts, ignore_index=True)
      .dropna(subset=["user_id", "ts"])
)

print("전체 활동 로그 행 수:", len(activity_log))

/var/folders/v1/k2x8lm750mbb_z0sj197vzrh0000gn/T/ipykernel_96123/1990533544.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat(activity_parts, ignore_index=True)


전체 활동 로그 행 수: 36748466


In [67]:

# ====== 3) 리텐션 = 참여 이후(ts > eng_first_at) 활동 로그가 1개라도 있는 유저 ======
# - 참여 유저만 남기기
activity_eng = activity_log.merge(eng_first, on="user_id", how="inner")

# - "참여 이후" 조건
activity_eng = activity_eng[activity_eng["ts"] > activity_eng["eng_first_at"]]

# - 리텐션 유저
ret_users = activity_eng[["user_id"]].drop_duplicates()
ret_cnt = len(ret_users)

print("리텐션 유저 수(any log after engagement):", ret_cnt)


리텐션 유저 수(any log after engagement): 14487


In [68]:

# ====== (옵션) 리텐션 유저별 어떤 로그로 리텐션 됐는지 샘플 보기 ======
ret_sample = (
    activity_eng.sort_values(["user_id", "ts"])
                .groupby("user_id", dropna=True)
                .head(3)
)
ret_sample.head(30)


,user_id,ts,src,eng_first_at
664733,1000071,2023-05-08 21:10:49,friendrequest,2023-05-08 18:44:16
669086,1000071,2023-05-09 06:52:48,friendrequest,2023-05-08 18:44:16
669335,1000071,2023-05-09 07:22:21,friendrequest,2023-05-08 18:44:16
707198,1000076,2023-05-11 07:48:56,friendrequest,2023-05-10 21:11:00
744402,1000076,2023-05-13 03:53:06,friendrequest,2023-05-10 21:11:00
759009,1000076,2023-05-13 13:35:49,friendrequest,2023-05-10 21:11:00
348262,1000118,2023-05-18 15:55:13,friendrequest,2023-05-17 22:09:53
348263,1000118,2023-05-18 15:55:16,friendrequest,2023-05-17 22:09:53
348264,1000118,2023-05-18 15:55:17,friendrequest,2023-05-17 22:09:53
944817,1000149,2024-03-05 12:30:49,friendrequest,2023-09-25 16:54:54
